In [2]:
!python -m pip install tslearn
import pprint as pp
import pickle

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn import svm

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, ShuffleSplit
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score,confusion_matrix
from sklearn.preprocessing import KBinsDiscretizer
from tslearn.clustering import TimeSeriesKMeans

from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.pipeline import Pipeline

from sklearn.metrics import accuracy_score, precision_score, recall_score,\
                            make_scorer, fbeta_score, roc_curve, auc, \
                            roc_auc_score, precision_recall_curve, confusion_matrix

     |████████████████████████████████| 440kB 1.3MB/s eta 0:00:01


/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.cluster.k_means_ module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.cluster. Anything that cannot be imported from sklearn.cluster is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
df = pd.read_csv('UCI_Credit_Card 2.csv')

In [9]:
payments = df[['PAY_AMT1','PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']]

In [11]:
km = TimeSeriesKMeans(n_clusters=3, metric="euclidean", max_iter=5,random_state=0).fit(payments)

In [4]:
df.columns

Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default.payment.next.month'],
      dtype='object')

In [12]:
km.predict(payments)

/opt/anaconda3/lib/python3.7/site-packages/tslearn/utils.py:91: UserWarning: 2-Dimensional data passed. Assuming these are 30000 1-dimensional timeseries
  '{} 1-dimensional timeseries'.format(X.shape[0]))


array([0, 0, 0, ..., 0, 1, 0])

In [95]:
est = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='kmeans')
df['LIMIT_BAL'] = est.fit_transform(df[['LIMIT_BAL']])

In [113]:
df['PAY_6'].value_counts()

 0    16286
-1     5740
-2     4895
 2     2766
 3      184
 4       49
 7       46
 6       19
 5       13
 8        2
Name: PAY_6, dtype: int64

In [100]:
X = df.iloc[:,:-1]
y = df['default.payment.next.month']

In [106]:
defaulters = df[df['default.payment.next.month'] == 1]

In [108]:
defaulters.co

0.0    3995
1.0    1653
2.0     722
3.0     248
4.0      18
Name: LIMIT_BAL, dtype: int64

In [102]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=42)

In [103]:
classifiers = {'KNN': KNeighborsClassifier(), 
               'LR': LogisticRegression(penalty = 'l1', max_iter = 10000, solver = 'liblinear'), 
               'RF': RandomForestClassifier(max_depth = 3), 
               'SVM': svm.SVC()
              }

samplers = {'sampling': [None, 
                         SMOTE(random_state = 42), 
                         ADASYN(random_state = 42)]
           }

results_dict = {}

# #n_splits = 1 in ShuffleSplits allows us to avoid cross-validation
# #cross-validating with n_jobs = -1 with sampling is not possible with pipeline
ss = ShuffleSplit(n_splits = 1)


for classifier_name, classifier in classifiers.items():
    steps = [('sampling', None),
             (classifier_name, classifier)]
    
    model = Pipeline(steps)
    
    grid = GridSearchCV(model, samplers, cv = ss, 
                        n_jobs = -1, scoring = 'roc_auc', verbose = True)

    grid.fit(X_train, y_train)

    results = grid.cv_results_
    
    scores = list(zip(results['params'], results['mean_test_score']))
    scores = sorted(scores, key = lambda x:x[1], reverse = True)

    results_dict[classifier_name] = scores

Fitting 1 folds for each of 3 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    5.4s finished


Fitting 1 folds for each of 3 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    2.0s finished


Fitting 1 folds for each of 3 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    2.5s finished


Fitting 1 folds for each of 3 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   48.7s finished


In [104]:
sampling_results = results_dict
pp.pprint(sampling_results)

{'KNN': [({'sampling': SMOTE(random_state=42)}, 0.5935283367431283),
         ({'sampling': ADASYN(random_state=42)}, 0.5897038369763554),
         ({'sampling': None}, 0.5877791813940035)],
 'LR': [({'sampling': None}, 0.7247687295692588),
        ({'sampling': SMOTE(random_state=42)}, 0.692732771464787),
        ({'sampling': ADASYN(random_state=42)}, 0.6903158286819809)],
 'RF': [({'sampling': None}, 0.7441225926979698),
        ({'sampling': ADASYN(random_state=42)}, 0.7371518980672336),
        ({'sampling': SMOTE(random_state=42)}, 0.736865121838499)],
 'SVM': [({'sampling': ADASYN(random_state=42)}, 0.6675946523421997),
         ({'sampling': SMOTE(random_state=42)}, 0.6665021578962014),
         ({'sampling': None}, 0.592328228679883)]}


In [105]:
classifiers = {'KNN': KNeighborsClassifier(), 
               'LR': LogisticRegression(penalty = 'l1', max_iter = 10000, solver = 'liblinear'), 
               'RF': RandomForestClassifier(max_depth = 3), 
               'SVM': svm.SVC()
              }

param_dict = {'KNN': {'KNN__n_neighbors': [3, 5, 7]}, 
              'LR': {'LR__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}, 
              'RF': {'RF__n_estimators': [100, 250, 500, 1000], 
                     'RF__max_depth': [3, 4, 5]},
              'SVM': {'SVM__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
             }

results_dict = {}

kf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)

for classifier_name, classifier in classifiers.items():
    
    #sampling is the first step, then classifying
    #'sampling' is None because no sampling was best
    #if 'sampling' is not None, change n_jobs in GridSearchCV to 1
    steps = [('sampling', None),
             (classifier_name, classifier)]
    
    model = Pipeline(steps)
    
    #fetch the parameters from param_dict
    params = param_dict[classifier_name]
    
    #scoring is changed to roc_auc score
    #we specify return_train_score = True to 
    #compare train and test to check for overfitting
    #if 'sampling' is not None, change n_jobs in GridSearchCV to 1
    #this is to avoid conflict among the workers 
    #due to sampling and modeling the training set, which changes across folds
    grid = GridSearchCV(model, params, cv = kf, return_train_score = True,
                        n_jobs = -1, scoring = 'roc_auc', verbose = True)

    grid.fit(X_train, y_train)

    results = grid.cv_results_
    
    scores = list(zip(results['params'], results['mean_train_score'], results['mean_test_score']))
    scores = sorted(scores, key = lambda x:x[2], reverse = True)

    results_dict[classifier_name] = scores

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:   14.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=-1)]: Done  35 out of  35 | elapsed:    4.4s finished


Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  2.2min finished


Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  35 out of  35 | elapsed: 16.1min finished


In [72]:
pp.pprint(results_dict)

{'KNN': [({'KNN__n_neighbors': 7}, 0.7987196829215204, 0.6107601504510046),
         ({'KNN__n_neighbors': 5}, 0.8298500560347524, 0.5998652638365873),
         ({'KNN__n_neighbors': 3}, 0.8830195829419442, 0.5822575005123554)],
 'LR': [({'LR__C': 0.01}, 0.6439141874016846, 0.6423485467115797),
        ({'LR__C': 100}, 0.6439141561292328, 0.6423485467115797),
        ({'LR__C': 1000}, 0.6439140801818495, 0.6423484752601439),
        ({'LR__C': 1}, 0.6439142231416298, 0.6423483323572721),
        ({'LR__C': 0.1}, 0.6439141650642191, 0.6423481894544003),
        ({'LR__C': 10}, 0.6439143616339169, 0.6423479751000927),
        ({'LR__C': 0.001}, 0.6439140623118769, 0.6423479036486568)],
 'RF': [({'RF__max_depth': 5, 'RF__n_estimators': 1000},
         0.787567700731761,
         0.7756750050826927),
        ({'RF__max_depth': 5, 'RF__n_estimators': 500},
         0.7874918990388309,
         0.7755071781457153),
        ({'RF__max_depth': 5, 'RF__n_estimators': 250},
         0.7873545390

In [76]:
def plot_roc_curves(classifiers, X_train, y_train, X_test, y_test, sampler = None):
    """
        Function for plotting roc curves of classifiers for comparison
        
        :param classifiers: dictionary of classifiers
        :param sampler: sampling method to use e.g. SMOTE
        
        :returns fpr: an array of false positive rate values from roc_curve
        :returns tpr: an array of true positive rate values from roc_curve
        :returns thresholds: an array of threshold values from roc_curve
        :returns roc_auc: roc_auc scores for each classifier in classifiers
    """

    fpr, tpr, thresholds = {}, {}, {}
    roc_auc = {}


    for classifier_name, classifier in classifiers.items():

        #set n_jobs = -1 for faster performance
        classifier = classifier.set_params(n_jobs = -1)

        if sampler:
            sampler.set_params(random_state = 42, n_jobs = -1)

        steps = [('sampling', sampler),
                 (classifier_name, classifier)]

        model = Pipeline(steps)

        model.fit(X_train, y_train)

        y_proba = model.predict_proba(X_test)[:,1]

        fpr[classifier_name], tpr[classifier_name], thresholds[classifier_name] = roc_curve(y_test, y_proba)

        roc_auc[classifier_name] = round(auc(fpr[classifier_name], tpr[classifier_name]), 3)

        print(classifier_name + ' roc_auc score: ' + str(roc_auc[classifier_name]))


    plt.figure(figsize=(8, 8))

    for classifier_name, classifier in classifiers.items():
        sns.lineplot(fpr[classifier_name], tpr[classifier_name], err_style = None);

    midline_points = np.arange(0, 1.05, 0.05) 

    plt.plot(midline_points, midline_points, linestyle ='--', color = 'black');
    plt.title("ROC Curves for Different Classification Models", y =1.05, fontsize = 16);
    plt.xlabel('FPR', fontsize = 12, x = 1.05);
    plt.ylabel('TPR', fontsize = 12, rotation =0, y = 1.05);
    plt.legend(list(zip(classifiers.keys(), roc_auc.values())));

    return fpr, tpr, thresholds, roc_auc